#### TODO:
- Algo seems to only be off by $100, but that's not much in the crypto world.
- Need to figure out how to make this more useful. 
- Next price point isn't bad, but what's the start? I can make a decent model. Now what?
    - Figure out how to generate the same data, but now estimate it for the next day.

# Use lagged values to predict current price to forecast price
- Notes from this video: https://www.youtube.com/watch?v=xaIA83x5Icg
 - Create separate transformers for features on different scales
 - Use this video for building an initial LSTM model
- Use this video and it's subsequent one for time series forecasting for BTC using binance exchange data: https://www.youtube.com/watch?v=jR0phoeXjrc
    - Use shift to push a column down for matching previous data points to a forecasted point.
    - This video is showing how to prep data for a time series model.
    - This video is for how to take that preprocessed data and put into PyTorch for the LSTM model: https://www.youtube.com/watch?v=ODEGJ_kh2aA&t=107s
         - 

In [171]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import os

In [172]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse

In [173]:
import tensorflow as tf
from keras import layers

In [174]:
from keras import models

In [175]:
from keras import optimizers

In [176]:
files = os.listdir(path='.')

In [177]:
crypto = pd.read_csv('ETH-USD.csv')

In [178]:
for col in crypto.columns.values:
    crypto.rename(columns={
        col: col.lower()
    }, inplace=True)

In [179]:
crypto.drop('adj close',axis=1, inplace=True)

In [180]:
future_estim = 5
output_label = 'next_close_'+str(future_estim)

In [181]:
crypto['next_close_'+str(future_estim)] = crypto['close'].shift(-1*future_estim)

In [183]:
window_size = 20

In [184]:
ma = []
upper = []
lower = []
for i in range(window_size):
    ma.append(np.nan)
    upper.append(np.nan)
    lower.append(np.nan)
for row in crypto.iterrows():
    if row[0] < window_size:
        continue
#     print(row[0])
    mean = np.mean(crypto.iloc[(row[0]-window_size):row[0]]['close'])
    ma.append(mean)
    std = np.std(crypto.iloc[(row[0]-window_size):row[0]]['close'])
    upper.append((mean+std*2))
    lower.append((mean-std*2)) 

In [185]:
len(upper)

1827

In [186]:
crypto['ma20'] = ma
crypto['upper'] = upper
crypto['lower'] = lower

In [187]:
crypto

,date,open,high,low,close,volume,next_close_5,ma20,upper,lower
0,2016-07-12,10.447000,10.762000,10.425800,10.516100,1.112770e+07,11.1587,NaN,NaN,NaN
1,2016-07-13,10.506600,10.589500,10.403900,10.500500,6.715660e+06,11.0344,NaN,NaN,NaN
2,2016-07-14,10.596900,11.681900,10.596900,11.513200,2.362190e+07,11.6191,NaN,NaN,NaN
3,2016-07-15,11.499100,12.431300,11.490000,11.951600,2.683890e+07,12.4506,NaN,NaN,NaN
4,2016-07-16,11.919400,11.935900,11.592400,11.652800,1.044570e+07,12.6507,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1822,2021-07-08,2317.949219,2324.006836,2089.414307,2120.026367,2.318812e+10,NaN,2109.220972,2438.549066,1779.892877
1823,2021-07-09,2115.573975,2185.376221,2051.066650,2146.692383,2.302957e+10,NaN,2103.635632,2428.218001,1779.053263
1824,2021-07-10,2146.999756,2190.124023,2081.923584,2111.403564,1.758154e+10,NaN,2102.045300,2425.454436,1778.636165
1825,2021-07-11,2110.875732,2172.653320,2083.803711,2139.664795,1.470539e+10,NaN,2095.297253,2411.940954,1778.653553


In [320]:
training_cols = []
for col in crypto.columns.values:
    if 'next' not in col and 'date' not in col and 'adj' not in col:
        training_cols.append(col)

In [321]:
training_cols

['open', 'high', 'low', 'close', 'volume', 'ma20', 'upper', 'lower']

In [252]:
crypto.fillna(0, inplace=True)

In [348]:
def train_models(df, label,training_cols, time_steps):
    minmax = {}
    for col in training_cols:
        temp_minmax = MinMaxScaler()
        df[col] = temp_minmax.fit_transform(df[[col]])
        minmax.update({col: temp_minmax})
    X = df[training_cols]
    y = df[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
    output = compare_rg(X_train, y_train, X_test, y_test, time_steps)
    return output, minmax

In [349]:
def compare_rg(X_train, y_train, X_test, y_test, time_steps):
    classifiers = {
        'XGBR': XGBRegressor(),
        'NN': neural_net(X_train.shape, time_steps)
    }

    outputs = {
        'XGBR': [],
        'NN': []
    }
    for model in classifiers:
        if model == 'NN':            
            algo = classifiers[model]
            X_train, y_train = create_dataset(X_train, y_train, time_steps)
            X_test, y_test = create_dataset(X_test, y_test, time_steps)
            algo.fit(X_train, y_train, X_test, y_test)
        else:
            algo = classifiers[model]
            algo.fit(X_train, y_train)
            y_predict = algo.predict(X_test)
            print(" ")
            print(model, ': ',mse(y_test, y_predict))
        for output in outputs:
            if output == model:
                outputs[output] = algo
        print('')
    return outputs

In [367]:
#https://machinelearningmastery.com/use-features-lstm-networks-time-series-forecasting/
# https://www.kdnuggets.com/2018/11/keras-long-short-term-memory-lstm-model-predict-stock-prices.html
class neural_net:
    def __init__(self, shape_input, time_steps):
        self.shape_input = shape_input
        self.model = models.Sequential()
        self.batch_size = 32
        self.model.add(layers.Bidirectional(
            layers.LSTM(units=8, input_shape=(time_steps, shape_input[1]), return_sequences=False)
        ))
        self.model.add(layers.Dropout(0.2, name='dropout_0'))
        self.model.add(layers.Dense(40, activation='relu'))
        self.model.add(layers.Dense(1, activation='relu'))
        self.model.compile(optimizer='adam', loss='mean_squared_error')
#         print(self.model.summary())

    
    def fit(self, X_train, y_train, X_test, y_test):
        self.model.fit(x=X_train, y=y_train, batch_size = self.batch_size, epochs=50, shuffle=False, validation_split=0.1)
        evaluation = self.model.evaluate(X_test, y_test)
        print(evaluation)
    
    def predict(self, Xnew):
        yNew = self.model.predict_classes(Xnew)
        return yNew

In [351]:
def create_dataset(X, y, time_steps = 20):
    y = y.values
    Xs, ys = [] , []
    for i in range(len(X) - time_steps):
        v = X[i: (i+time_steps)]
        Xs.append(v.values)
        ys.append(y[(i+time_steps)])
    return np.array(Xs), np.array(ys)

In [368]:
output, minmax = train_models(crypto, output_label,training_cols, window_size)

 
XGBR :  6627.872553429373

Epoch 1/50
41/41 [==============================] - 1s 20ms/step - loss: 617420.9375 - val_loss: 884140.1250
Epoch 2/50
41/41 [==============================] - 0s 5ms/step - loss: 615106.3750 - val_loss: 877900.9375
Epoch 3/50
41/41 [==============================] - 0s 5ms/step - loss: 607134.2500 - val_loss: 865959.0625
Epoch 4/50
41/41 [==============================] - 0s 5ms/step - loss: 595600.3750 - val_loss: 850209.1875
Epoch 5/50
41/41 [==============================] - 0s 5ms/step - loss: 580911.5625 - val_loss: 831615.2500
Epoch 6/50
41/41 [==============================] - 0s 5ms/step - loss: 564461.1250 - val_loss: 811443.7500
Epoch 7/50
41/41 [==============================] - 0s 5ms/step - loss: 547118.2500 - val_loss: 790351.7500
Epoch 8/50
41/41 [==============================] - 0s 6ms/step - loss: 529318.1875 - val_loss: 768805.0000
Epoch 9/50
41/41 [==============================] - 0s 6ms/step - loss: 511954.8750 - val_loss: 747538.062

In [369]:
model = output['XGBR']

In [370]:
y_predict = model.predict(minmax.transform(crypto[training_cols]))

AttributeError: 'dict' object has no attribute 'transform'

In [ ]:
crypto['predicted_close'] = y_predict

In [ ]:
crypto = crypto.tail(100)

In [ ]:
y_true = crypto['next_close'].tolist()
y_predict = crypto['predicted_close'].tolist()
y_ma = crypto['ma20'].tolist()
y_upper = crypto['upper'].tolist()
y_lower = crypto['lower'].tolist()
x = crypto['date'].tolist()

In [ ]:
fig = plt.figure(figsize=(40,20))
plt.plot(x, y_predict, label='ETH')
plt.plot(x, y_true, label='ETH-Est')
plt.plot(x, y_ma, label='20 Day MA')
plt.plot(x, y_upper, label='Upper Bound')
plt.plot(x, y_lower, label='Lower Bound')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
training_cols

In [ ]:
base_url = 'https://api.pro.coinbase.com'

In [ ]:
products = '/products'

In [ ]:
book = '/book?level='

In [ ]:
stats = '/stats'

In [ ]:
#Only shows best bid and ask
level_1 = '1'
#Only shows top 50 bids and asks
level_2 = '2'
#Shows all bids and asks
level_3 = '3'

In [ ]:
curr1 = 'ETH'
curr2 = 'USD'
pair = '/' + curr1 + '-' + curr2

In [ ]:
response = requests.get(base_url+products+pair+stats).json()

In [ ]:
training_cols

In [ ]:
response

In [ ]:
to_eval = pd.DataFrame(response, index=[0])

In [ ]:
to_eval.drop('volume_30day', axis=1, inplace=True)

In [ ]:
to_eval.rename(columns={
    'last': 'close'
},inplace=True)

In [ ]:
to_eval['ma20'] = np.mean(crypto.iloc[(crypto.shape[0]-window_size-1): (crypto.shape[0]-1)]['ma20'])

In [ ]:
to_eval['upper'] = np.std(crypto.iloc[(crypto.shape[0]-window_size-1): (crypto.shape[0]-1)]['ma20'])*2 + np.mean(crypto.iloc[(crypto.shape[0]-window_size-1): (crypto.shape[0]-1)]['ma20'])

In [ ]:
to_eval['lower'] = np.mean(crypto.iloc[(crypto.shape[0]-window_size-1): (crypto.shape[0]-1)]['ma20']) - np.std(crypto.iloc[(crypto.shape[0]-window_size-1): (crypto.shape[0]-1)]['ma20'])*2 

In [ ]:
evald = minmax.transform(to_eval[training_cols])

In [ ]:
result = model.predict(evald)

In [ ]:
to_eval['predicted_close'] = result[0]

In [ ]:
crypto = crypto.append(to_eval)

In [ ]:
crypto

In [ ]:
crypto = crypto.reset_index()

In [ ]:
for row in crypto[['date', 'next_close', 'predicted_close']].iterrows():
    print(row[1]['date'])
    close = np.round(row[1]['next_close'])
    predicted_close = np.round(row[1]['predicted_close'])
    print('Next Close: ', close)
    print('Predicted Close: ', predicted_close)
    print('The diff: ', (close - predicted_close))
    print(' ')